In [1]:
from optbinning import OptimalBinning
from collections import defaultdict

from os import listdir
from os.path import isfile, join
from datetime import datetime, timedelta
import time
import sys

import pandas as pd
import numpy as np
import random

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

from optbinning import OptimalBinning
import keras



from catboost import CatBoostClassifier, Pool, cv

import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


from bayes_opt import BayesianOptimization
import catboost

In [2]:
import scipy as sc

In [3]:

i = 'dataset1.csv'
df = pd.read_csv(i, sep=';', low_memory=False, error_bad_lines=False)
# delete important variables    
#for col in ['x_24', 'x_10', 'x_38', 'x_6', 'x_17', 'x_12']:
#    df = df.drop(col, axis=1)
    
df_view = df.copy()

In [4]:
#df['x_15'].value_counts()

In [5]:
#adding variables with many levels following a behaviour that explains the target


In [6]:
print(df.shape)
#df.columns
df_types = df_view.dtypes
df_types.value_counts()

(99976, 43)


int64      20
float64    19
object      3
bool        1
dtype: int64

In [7]:
categorical_cols = df_types[df_types=='object']
categorical_cols = list(categorical_cols.index)
categorical_cols = [x for x in categorical_cols if x!='uuid']
categorical_cols

['x_13', 'x_14', 'x_18']

In [8]:
for i in categorical_cols:
    print(len(set(df[i])))

57
12
8


In [9]:
random.seed(1)
df['random'] = np.random.randint(1, df.shape[0], df.shape[0])/df.shape[0]
porc_train=0.6
porc_test=0.2
df['flag_train'] = 0 #train
df['flag_train'] = ((df['random'] > porc_train)).astype('int') #eval 1
df.loc[df['random'] > (1-porc_test),'flag_train']=2 #test

df.loc[df['default'].isnull(), 'flag_train'] = 9 #to_predict
df.flag_train.value_counts()

0    53986
1    18018
2    17972
9    10000
Name: flag_train, dtype: int64

In [10]:
df_view = df.copy()
df_view= df_view[df_view.default.isnull()==False]

table_bivar_cat = dict()
groups_bivar_cat = defaultdict(dict)

for var in categorical_cols :
    print('var: ' + str(var) )
    if var in categorical_cols:
        df_view.loc[df_view[var].isnull(),var]='SD'
        df_view.loc[df_view[var].isna(),var]='SD'


    y = df_view[(df_view['flag_train'] ==1)].default 
    X = df_view[(df_view['flag_train'] ==1)][var].values

    try: 
        optb = OptimalBinning(name = var, dtype="categorical",solver="cp", max_pvalue=.2 )
        optb.fit(X, y)

        tbla = optb.binning_table.build()
        tbla['Bin'] = tbla.Bin.apply( lambda x: str(x)).astype('string')
        tbla = tbla[tbla.Bin.isin(['Missing', 'Special', ''])==False]
        #display(tbla)
        table_bivar_cat[var] = tbla.iloc[:,:6]

        #saves cuts in table
       
        groups = list(zip(optb.splits, np.arange(0,len(optb.splits))))
       
        for group in groups:
            for elem in group[0]:
                groups_bivar_cat[var][elem]= group[1]

        # nombre=str(var)+'_logit'
        # p=optb.transform(df_cons0[var], metric="event_rate")
        # df_cons0[nombre] = np.log(p/(1-p))

    except:
        print(var)
        print('Unexpected error:', sys.exc_info()[0])

        pass

var: x_13
var: x_14
var: x_18


In [11]:
def map_cat_groups(df, groups_bivar_cat):
    for var in groups_bivar_cat:
        cat_name = str(var)+'_cat'
        df[cat_name] = df[var].map(groups_bivar_cat[var])
        imputation_value = df.loc[~df[cat_name].isnull(),cat_name].median()
        df.loc[df[cat_name].isnull(),cat_name] = imputation_value
        df[cat_name] = df[cat_name].astype(int)
    return df

df_view = map_cat_groups(df_view, groups_bivar_cat)

df = map_cat_groups(df, groups_bivar_cat)

In [12]:
limit_br=0.01*4
print('Print only if 1 value has positive rate greater than '+str(limit_br))
for key, value in table_bivar_cat.items():
  
  if (value.shape[0]>2) and (sum(value['Event rate']>limit_br)>0):
    print (key[0])
    print (key[1:])
    #print( value.to_string(index=False))
    display( value)
    print('\n ')

Print only if 1 value has positive rate greater than 0.04
x
_13


,Bin,Count,Count (%),Non-event,Event,Event rate
0,['factor_0' 'factor_7' 'factor_56' 'factor_51'...,2159,0.119825,2150,9,0.004169
1,['factor_52' 'factor_19' 'factor_21' 'factor_1...,8551,0.474581,8489,62,0.007251
2,['factor_35' 'factor_50' 'factor_44' 'factor_1...,3389,0.188090,3328,61,0.017999
3,['factor_46' 'factor_24' 'factor_33' 'factor_6...,2963,0.164447,2881,82,0.027675
4,['factor_15' 'factor_38' 'factor_30' 'factor_5...,956,0.053058,907,49,0.051255


## autoencoders

In [13]:
df_encod = df.copy()
dum_df = pd.DataFrame(index = df.index)

for var in categorical_cols:
    print(var)
    #print('levels ' + str(levels_var))
    X = pd.DataFrame(pd.get_dummies(df[var]) )
    X = pd.DataFrame(X)
    X.columns = [str(var) +'_' +str(level_name) for level_name  in X.columns]
  
    dum_df = dum_df.join(X)

x_13
x_14
x_18


In [14]:
dum_df

,x_13_factor_0,x_13_factor_1,x_13_factor_10,x_13_factor_11,x_13_factor_12,x_13_factor_13,x_13_factor_14,x_13_factor_15,x_13_factor_16,x_13_factor_17,...,x_14_factor_8,x_14_factor_9,x_18_factor_0,x_18_factor_1,x_18_factor_2,x_18_factor_3,x_18_factor_4,x_18_factor_5,x_18_factor_6,x_18_factor_7
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99972,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
99973,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
99974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [15]:

cols  = [col for col in df.columns if 'x_' in col and 'cat' not in col]
cols = list(set(cols) - set(categorical_cols))
dum_df = dum_df.join(df[cols])

for col in dum_df.columns:
    if sum(dum_df[col].isnull())>0:
    #print(col)
        dum_df.loc[dum_df[col].isnull(), col] = np.mean(dum_df.loc[dum_df[col].isnull()==False, col])
    

dum_df['x_15'] = dum_df['x_15'].astype(int)    

In [16]:
for col in dum_df.columns:
    dum_df[col] = (dum_df[col]-dum_df[col].min())/(dum_df[col].max()-dum_df[col].min())

In [17]:
levels_var = dum_df.shape[1]
print(levels_var)
encoding_dim = 70 # max(int( np.floor (levels_var*.10 )),10)
#40  same result
#60  89-91 auc , 61 to 65 ks
input_img = keras.Input(shape=(levels_var,))

#encoded = keras.layers.Dense(encoding_dim, activation='relu')(input_img)
encoded = keras.layers.Dense(encoding_dim, activation='tanh')(input_img)

decoded = keras.layers.Dense(levels_var, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)

encoder = keras.Model(input_img, encoded) # This model maps an input to its encoded representation

decoder_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(decoder_input, decoder_layer(decoder_input))

autoencoder.compile(optimizer='adam', 
                    loss='binary_crossentropy')
autoencoder.fit(dum_df.loc[(df['flag_train'] ==0) ], 
                dum_df.loc[(df['flag_train'] ==0) ] ,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(dum_df.loc[(df['flag_train'] ==1) ], 
                                 dum_df.loc[(df['flag_train'] ==1) ])
                )

115
Epoch 1/100
211/211 [==============================] - 17s 7ms/step - loss: 0.3950 - val_loss: 0.1311
Epoch 2/100
211/211 [==============================] - 1s 4ms/step - loss: 0.1242 - val_loss: 0.1058
Epoch 3/100
211/211 [==============================] - 1s 5ms/step - loss: 0.1007 - val_loss: 0.0880
Epoch 4/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0846 - val_loss: 0.0777
Epoch 5/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0757 - val_loss: 0.0719
Epoch 6/100
211/211 [==============================] - 1s 6ms/step - loss: 0.0706 - val_loss: 0.0682
Epoch 7/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0672 - val_loss: 0.0657
Epoch 8/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0649 - val_loss: 0.0638
Epoch 9/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0632 - val_loss: 0.0624
Epoch 10/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0618 - v

211/211 [==============================] - 1s 4ms/step - loss: 0.0533 - val_loss: 0.0536
Epoch 82/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0532 - val_loss: 0.0536
Epoch 83/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0535 - val_loss: 0.0536
Epoch 84/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0534 - val_loss: 0.0536
Epoch 85/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0534 - val_loss: 0.0536
Epoch 86/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0533 - val_loss: 0.0536
Epoch 87/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0534 - val_loss: 0.0536
Epoch 88/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0533 - val_loss: 0.0536
Epoch 89/100
211/211 [==============================] - 1s 4ms/step - loss: 0.0533 - val_loss: 0.0536
Epoch 90/100
211/211 [==============================] - 1s 5ms/step - loss: 0.0533 - val_loss: 

In [18]:
encoded = encoder.predict(dum_df)
print(encoded.shape)
encoded_df =  pd.DataFrame(encoded)
encoded_df.columns

(99976, 70)


RangeIndex(start=0, stop=70, step=1)

In [19]:
cols_keep =['default', 'flag_train', 'id']
#encoded_df.columns = ['encoded_'+str(var) + '_lev'+ str(level_name) for level_name  in encoded_df.columns]
df_encod = df_encod[cols_keep].join(encoded_df)

In [20]:
decoded_imgs = decoder.predict(encoded)
#X.iloc[0,:] 
for i in range(10):
    print(decoded_imgs[i][dum_df.iloc[i,:] == 1]) #only makes sense for binaries


[0.9999316  0.9999958  0.99998975 1.        ]
[0.99997985 0.99999946 0.99999535 1.        ]
[0.99999714 0.9999993  0.99998486 1.        ]
[0.9999994 0.9999996 0.9999662 1.       ]
[0.9999019  0.99988973 0.999993   0.9999979 ]
[0.99965537 0.9999303  0.99999785]
[0.9999516  0.9999894  0.99994934 0.9999959 ]
[0.999931 0.999859 0.99999  1.      ]
[0.9999889 0.999999  0.9998748 1.       ]
[0.99998987 0.99999905 0.9999708  1.        ]


In [21]:
df_encod.shape

(99976, 73)

In [22]:
df_encod.to_csv('df_encod.csv', index=False)